#### Importing Required Python Packages

In [2]:
!pip install html_table_extractor

In [3]:
import pandas as pd
import numpy as np
import os
import random
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import time
from html_table_extractor.extractor import Extractor

#### Return the soup object for any given URL

In [4]:
def get_html(url):    
    try:
        ### Get the response for the given input url
        response=requests.get(url)
        soup=BeautifulSoup(response.content)   ### Get the soup object for the input url
        return soup
        
    #### Return the url and error for logs
    except Exception as e:
        return ''

#### Extracting the tables from soup object

In [5]:
    #### Iterating through all the rows in the table

def table_extraction_naics(soup,class_name):
    #'companyDetail topCompanyDetail'
    ### Source table data extraction
    #data_df = table_extraction(soup,types = 'default',class_name='companyDetail topCompanyDetail')  ### Tables extracted from the given url
    ### Iterate until there are br tags    
    # print('Total Number of breaks:', len(soup_.findAll('br')))
    # while len(soup_.findAll('br')):
    #     soup_.br.replace_with("\n")  ### Replaces the line break tag with new line character

        table = soup.find('table', {'class': class_name})    ### Get all the tables for the given class from a soup object
        table_data = []   ### For storing the row level text data for the entire table 
        anchor_data = []  ### For storing the row level link data for the entire table 
        table_headers=[]
        if table!=None:
            table_headers = [header.getText().replace(':','').strip() for header in table.findAll('th')]    #### Get the table headers from the given table    
            rows = table.findAll('tr')     ### Get all the row level data from table

            table_data = []   ### For storing the row level text data for the entire table 
            anchor_data = []  ### For storing the row level link data for the entire table 
            ids = []          ### For storing index where the links are identified 

            for tr in rows:

                cols = tr.findAll('td')  #### Get the various columns or data present in each row

                table_rows = []   ### Store the content/text of the table 
                anchor_tag = []   ### Store the link/anchor tags of the table

                ### Iterate through the columns for each row to get the text and anchor tags
                for idx,col in enumerate(cols):
                    table_rows.append(col.getText())    ### Get the text level data present in each row and each column

                    if col.findAll('a'):               ### Check if the anchor or link information is present in the html
                        anchor_tag.append(col.find('a').get('href'))     ### If present then get the href link
                        ids.append(idx)                ### Get the indices where the links are found

                if len(table_rows)>0:
                    table_data.append(table_rows)    #### Append the row level content/text to master list

                if len(anchor_tag)>0:
                    anchor_data.append(anchor_tag)   #### Append the row level links/href to master list
        return(table_data,anchor_data,table_headers)

In [6]:
df_naics=pd.read_excel("/content/Naics_with_desc (1).xlsx")

In [7]:
df_naics['desc']=''
df_naics['cross_reference']=''

In [8]:
concordance_table=pd.DataFrame()
top_comp_table=pd.DataFrame()

In [9]:
begin = time.time()
for j in range(0,df_naics.shape[0]): #df_naics.shape[0]
    i=df_naics['naics_6digit_1_code'][j]
    html_link="https://www.naics.com/naics-code-description/?code="+str(i)
    print(html_link)
    soup = get_html(html_link)    ### Get the soup object for the given url 
    if soup.find('div', {'class': "entry-content"})!=None:
        df_naics['desc'][j]=str(soup.find('div', {'class': "entry-content"}).find("p"))
        
    if soup.find('ul', {'class': "crossreference"})!=None:
        df_naics['cross_reference'][j]=str(soup.find('ul', {'class': "crossreference"}))

    temp=table_extraction_naics(soup,"concordance table table-striped")
    temp_df1=pd.DataFrame(temp[0],columns=temp[2])
    temp=table_extraction_naics(soup,"topCompanies table table-striped")
    temp_df2=pd.concat([pd.DataFrame(temp[0],columns=['Company_Name','City','State']),pd.DataFrame(temp[1],columns=['Link'])],axis=1)
    temp_df1['naics_6digit_1_code']=i
    temp_df2['naics_6digit_1_code']=i
    concordance_table=concordance_table.append(temp_df1)
    top_comp_table=top_comp_table.append(temp_df2)

https://www.naics.com/naics-code-description/?code=111110


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


https://www.naics.com/naics-code-description/?code=111120
https://www.naics.com/naics-code-description/?code=111130
https://www.naics.com/naics-code-description/?code=111140
https://www.naics.com/naics-code-description/?code=111191
https://www.naics.com/naics-code-description/?code=111199
https://www.naics.com/naics-code-description/?code=111211
https://www.naics.com/naics-code-description/?code=111219
https://www.naics.com/naics-code-description/?code=111331
https://www.naics.com/naics-code-description/?code=111332
https://www.naics.com/naics-code-description/?code=111335
https://www.naics.com/naics-code-description/?code=111336
https://www.naics.com/naics-code-description/?code=111339
https://www.naics.com/naics-code-description/?code=111411
https://www.naics.com/naics-code-description/?code=111419
https://www.naics.com/naics-code-description/?code=111421
https://www.naics.com/naics-code-description/?code=111422
https://www.naics.com/naics-code-description/?code=111920
https://www.na

In [10]:
import re
def clean_text(x):
    x1=re.sub("\<[^<>]*\>", "", x)
    return(x1)
def split_text(x):
    x1=x.split("<u>Cross-References")[0]
    return(x1)

In [11]:
df_naics['cleaned_desc']=df_naics['desc'].apply(lambda x: clean_text(split_text(x)))

In [12]:
df_naics['cleaned_cross_ref']=df_naics['cross_reference'].apply(lambda x: clean_text(x))

In [13]:
df_naics.head()

,naics_6digit_1_code,cleaned_desc,cleaned_cross_ref,desc,cross_reference
0,111110,This industry comprises establishments primari...,Establishments engaged in growing soybeans in ...,<p>This industry comprises establishments prim...,"<ul class=""crossreference""><li>Establishments ..."
1,111120,This industry comprises establishments primari...,Growing soybeans--are classified in Industry 1...,<p>This industry comprises establishments prim...,"<ul class=""crossreference""><li>Growing soybean..."
2,111130,This industry comprises establishments primari...,Establishments primarily engaged in growing fr...,<p>This industry comprises establishments prim...,"<ul class=""crossreference""><li>Establishments ..."
3,111140,This industry comprises establishments primari...,Establishments growing wheat in combination wi...,<p>This industry comprises establishments prim...,"<ul class=""crossreference""><li>Establishments ..."
4,111191,This U.S. industry comprises establishments en...,Establishments engaged in growing one grain (o...,<p>This U.S. industry comprises establishments...,"<ul class=""crossreference""><li>Establishments ..."


In [ ]:
with pd.ExcelWriter('NAICS_output.xlsx') as writer:  
    df_naics.to_excel(writer, sheet_name='Sheet1')
    concordance_table.to_excel(writer, sheet_name='Sheet2')
    top_comp_table.to_excel(writer, sheet_name='Sheet3')

In [ ]:
sic_table1=pd.read_excel("sic_class.xlsx",dtype={'Code': 'str'})

In [ ]:
sic_table1.head()

,Code,Industry Title,Number of Business Establishments,Link,link2
0,0111,wheat,7338,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...
1,0112,Rice,945,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...
2,0115,Corn,28047,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...
3,0116,Soybeans,4656,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...
4,0119,"Cash Grains, Nec",18972,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...


In [ ]:
sic_table1['Desc']=""

In [ ]:
t1=pd.DataFrame()
t2=pd.DataFrame()

In [ ]:
for i in range(0,sic_table1.shape[0]):
    html_link=sic_table1['Link'][i]
    print(html_link)
    soup = get_html(html_link)

    sic_table1['Desc'][i]=clean_text(str(soup.find('div', {'class': "entry-content"})).split("</h6>")[1].split("<table")[0]).strip()

    temp=table_extraction_naics(soup,"table table-striped")
    temp_df1=pd.DataFrame(temp[0],columns=temp[2])
    temp_df1['SIC_CODE']=sic_table1['Code'][i]
    t1=t1.append(temp_df1)

    temp=table_extraction_naics(soup,"sixDigitTable")
    temp_df2=pd.DataFrame(temp[0])
    temp_df2['SIC_CODE']=sic_table1['Code'][i]
    t2=t2.append(temp_df2)

https://www.naics.com/sic-industry-description/?code=0111


C:\Users\harish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


https://www.naics.com/sic-industry-description/?code=0112
https://www.naics.com/sic-industry-description/?code=0115
https://www.naics.com/sic-industry-description/?code=0116
https://www.naics.com/sic-industry-description/?code=0119
https://www.naics.com/sic-industry-description/?code=0131
https://www.naics.com/sic-industry-description/?code=0132
https://www.naics.com/sic-industry-description/?code=0133
https://www.naics.com/sic-industry-description/?code=0134
https://www.naics.com/sic-industry-description/?code=0139
https://www.naics.com/sic-industry-description/?code=0161
https://www.naics.com/sic-industry-description/?code=0171
https://www.naics.com/sic-industry-description/?code=0172
https://www.naics.com/sic-industry-description/?code=0173
https://www.naics.com/sic-industry-description/?code=0174
https://www.naics.com/sic-industry-description/?code=0175
https://www.naics.com/sic-industry-description/?code=0179
https://www.naics.com/sic-industry-description/?code=0181
https://www.na

C:\Users\harish\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


https://www.naics.com/sic-industry-description/?code=0241
https://www.naics.com/sic-industry-description/?code=0251
https://www.naics.com/sic-industry-description/?code=0252
https://www.naics.com/sic-industry-description/?code=0253
https://www.naics.com/sic-industry-description/?code=0254
https://www.naics.com/sic-industry-description/?code=0259
https://www.naics.com/sic-industry-description/?code=0271
https://www.naics.com/sic-industry-description/?code=0272
https://www.naics.com/sic-industry-description/?code=0273
https://www.naics.com/sic-industry-description/?code=0279
https://www.naics.com/sic-industry-description/?code=0291
https://www.naics.com/sic-industry-description/?code=0711
https://www.naics.com/sic-industry-description/?code=0721
https://www.naics.com/sic-industry-description/?code=0722
https://www.naics.com/sic-industry-description/?code=0723
https://www.naics.com/sic-industry-description/?code=0724
https://www.naics.com/sic-industry-description/?code=0741
https://www.na

https://www.naics.com/sic-industry-description/?code=2341
https://www.naics.com/sic-industry-description/?code=2342
https://www.naics.com/sic-industry-description/?code=2353
https://www.naics.com/sic-industry-description/?code=2361
https://www.naics.com/sic-industry-description/?code=2369
https://www.naics.com/sic-industry-description/?code=2371
https://www.naics.com/sic-industry-description/?code=2381
https://www.naics.com/sic-industry-description/?code=2384
https://www.naics.com/sic-industry-description/?code=2385
https://www.naics.com/sic-industry-description/?code=2386
https://www.naics.com/sic-industry-description/?code=2387
https://www.naics.com/sic-industry-description/?code=2389
https://www.naics.com/sic-industry-description/?code=2391
https://www.naics.com/sic-industry-description/?code=2392
https://www.naics.com/sic-industry-description/?code=2393
https://www.naics.com/sic-industry-description/?code=2394
https://www.naics.com/sic-industry-description/?code=2395
https://www.na

https://www.naics.com/sic-industry-description/?code=3221
https://www.naics.com/sic-industry-description/?code=3229
https://www.naics.com/sic-industry-description/?code=3231
https://www.naics.com/sic-industry-description/?code=3241
https://www.naics.com/sic-industry-description/?code=3251
https://www.naics.com/sic-industry-description/?code=3253
https://www.naics.com/sic-industry-description/?code=3255
https://www.naics.com/sic-industry-description/?code=3259
https://www.naics.com/sic-industry-description/?code=3261
https://www.naics.com/sic-industry-description/?code=3262
https://www.naics.com/sic-industry-description/?code=3263
https://www.naics.com/sic-industry-description/?code=3264
https://www.naics.com/sic-industry-description/?code=3269
https://www.naics.com/sic-industry-description/?code=3271
https://www.naics.com/sic-industry-description/?code=3272
https://www.naics.com/sic-industry-description/?code=3273
https://www.naics.com/sic-industry-description/?code=3274
https://www.na

https://www.naics.com/sic-industry-description/?code=3621
https://www.naics.com/sic-industry-description/?code=3624
https://www.naics.com/sic-industry-description/?code=3625
https://www.naics.com/sic-industry-description/?code=3629
https://www.naics.com/sic-industry-description/?code=3631
https://www.naics.com/sic-industry-description/?code=3632
https://www.naics.com/sic-industry-description/?code=3633
https://www.naics.com/sic-industry-description/?code=3634
https://www.naics.com/sic-industry-description/?code=3635
https://www.naics.com/sic-industry-description/?code=3639
https://www.naics.com/sic-industry-description/?code=3641
https://www.naics.com/sic-industry-description/?code=3643
https://www.naics.com/sic-industry-description/?code=3644
https://www.naics.com/sic-industry-description/?code=3645
https://www.naics.com/sic-industry-description/?code=3646
https://www.naics.com/sic-industry-description/?code=3647
https://www.naics.com/sic-industry-description/?code=3648
https://www.na

https://www.naics.com/sic-industry-description/?code=5063
https://www.naics.com/sic-industry-description/?code=5064
https://www.naics.com/sic-industry-description/?code=5065
https://www.naics.com/sic-industry-description/?code=5072
https://www.naics.com/sic-industry-description/?code=5074
https://www.naics.com/sic-industry-description/?code=5075
https://www.naics.com/sic-industry-description/?code=5078
https://www.naics.com/sic-industry-description/?code=5082
https://www.naics.com/sic-industry-description/?code=5083
https://www.naics.com/sic-industry-description/?code=5084
https://www.naics.com/sic-industry-description/?code=5085
https://www.naics.com/sic-industry-description/?code=5087
https://www.naics.com/sic-industry-description/?code=5088
https://www.naics.com/sic-industry-description/?code=5091
https://www.naics.com/sic-industry-description/?code=5092
https://www.naics.com/sic-industry-description/?code=5093
https://www.naics.com/sic-industry-description/?code=5094
https://www.na

https://www.naics.com/sic-industry-description/?code=7373
https://www.naics.com/sic-industry-description/?code=7374
https://www.naics.com/sic-industry-description/?code=7375
https://www.naics.com/sic-industry-description/?code=7376
https://www.naics.com/sic-industry-description/?code=7377
https://www.naics.com/sic-industry-description/?code=7378
https://www.naics.com/sic-industry-description/?code=7379
https://www.naics.com/sic-industry-description/?code=7381
https://www.naics.com/sic-industry-description/?code=7382
https://www.naics.com/sic-industry-description/?code=7383
https://www.naics.com/sic-industry-description/?code=7384
https://www.naics.com/sic-industry-description/?code=7389
https://www.naics.com/sic-industry-description/?code=7513
https://www.naics.com/sic-industry-description/?code=7514
https://www.naics.com/sic-industry-description/?code=7515
https://www.naics.com/sic-industry-description/?code=7519
https://www.naics.com/sic-industry-description/?code=7521
https://www.na

In [ ]:
sic_table1

,Code,Industry Title,Number of Business Establishments,Link,link2,Desc
0,0111,wheat,7338,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Establishments primarily engaged in the produc...
1,0112,Rice,945,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Establishments primarily engaged in the produc...
2,0115,Corn,28047,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Establishments primarily engaged in the produc...
3,0116,Soybeans,4656,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Establishments primarily engaged in the produc...
4,0119,"Cash Grains, Nec",18972,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Establishments primarily engaged in the produc...
...,...,...,...,...,...,...
865,9641,Regulation of Agricultural Marketing,6275,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Government establishments primarily engaged in...
866,9651,"Regulation, Miscellaneous Commercial Sectors",3583,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Government establishments primarily engaged in...
867,9661,Space Research and Technology,185,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,Government establishments primarily engaged in...
868,9711,National Security,14640,https://www.naics.com/sic-industry-description...,https://www.naics.com/sic-industry-description...,"Establishments of the armed forces, including ..."


In [ ]:
with pd.ExcelWriter('SIC_output.xlsx') as writer:  
    sic_table1.to_excel(writer, sheet_name='Sheet1')
    t1.to_excel(writer, sheet_name='Sheet2')
    t2.to_excel(writer, sheet_name='Sheet3')